In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data=pd.read_csv('../input/adult-census-income/adult.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.columns=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','country','salary']

In [ ]:
data.isnull().sum()

In [ ]:

data['workclass'].unique()

In [ ]:
data.isin([' ?']).sum()

In [ ]:
data=data.replace(' ?',np.nan)

In [ ]:
print('Continous variables in the dataset are: ')
cont = []
for i in data.columns:
    if(data[i].dtype=='int64'):
        
        cont.append(i)
        print(i,end=',')
print('\n\nCategorical variable in the dataset are: ')
catg=[]
for i in data.columns:
    if data[i].dtype=='O':
        
        catg.append(i)
        print(i,end=', ')

In [ ]:
data.head()

In [ ]:
data.salary.describe()

In [ ]:
for i in cont:
    sns.distplot(data[i])
    plt.show()

In [ ]:
for i in catg:
    sns.countplot(data[i])
    plt.show()

In [ ]:
data['marital-status'].unique()

In [ ]:
data.info()

In [ ]:
sns.relplot(data=data, x="age", y="capital-loss",hue='marital-status', kind="line")

In [ ]:
data.info()

In [ ]:
import seaborn as sns
sns.barplot(x = data['education'], y = data['education-num'])

In [ ]:
import seaborn as sns
sns.barplot(x = data['sex'], y = data['capital-loss'])

In [ ]:
data['education-num'].unique()

In [ ]:
sns.boxplot(data['capital-gain'])
plt.show()


In [ ]:
# data sets with high kurtosis tend to have heavy tails, or outliers. 
# Data sets with low kurtosis tend to have light tails, or lack of outliers
stats.kurtosis(data['capital-gain'])

In [ ]:

ind = data[(data['capital-gain'] < 10000)].index

In [ ]:
stats.kurtosis(data.drop(ind)['capital-gain'])

In [ ]:
sns.boxplot(data.drop(ind)['capital-gain'])

In [ ]:
data.drop(ind)['capital-gain'].shape

In [ ]:
cont=['age','education-num','capital-gain','capital-loss','hours-per-week']

In [ ]:
catg

In [ ]:

plt.figure(figsize=(10,7))
grp=data['workclass'].value_counts()
sns.barplot(grp.index,grp.values)


In [ ]:
data['workclass'].unique()

In [ ]:
grp.values

In [ ]:
data[data['salary']=='<=50K']['workclass'].value_counts()

In [ ]:
data.info()

In [ ]:
def catg_vis(dt,cg):
  plt.figure(figsize=(20,5))
  a=dt[dt['salary']=='<=50K'][cg].value_counts() # Below 50k salary
  b=dt[dt['salary']=='>50K'][cg].value_counts() # Above 50k salary
  plt.subplot(1,2,1)
  plt.title('Below 50K salary')
  sns.barplot(a.index,a.values)
  plt.subplot(1,2,2)
  plt.title('Above 50K salary')
  sns.barplot(b.index,b.values)
  plt.show()

In [ ]:
for i in catg:
  print(i,':\n')
  catg_vis(data,i)

In [ ]:
catg=['occupation','race','sex']


In [ ]:
data[cont+catg]

In [ ]:
# analyzing capital gain and capital loss
data['capital-gain'].describe()

In [ ]:
plt.hist(data['capital-gain'])
plt.show()

In [ ]:
stats.skew(data['capital-gain'])

In [ ]:
cont

In [ ]:
for i in range(2):
    cont.pop(2)

In [ ]:
cont

In [ ]:
print(cont)
print('\n',catg)

In [ ]:
data[cont+catg].head()

In [ ]:
le=LabelEncoder()
y=data['sex'].values
data['sext']=le.fit_transform(y)
y=data['occupation'].values
data['occupationt']=le.fit_transform(y)
y=data['race'].values
data['racet']=le.fit_transform(y)

data['salaryT']=le.fit_transform(data['salary'])

In [ ]:
sel_cols=['age','education-num','hours-per-week','occupationt','racet','sext']

In [ ]:
data[sel_cols].head()

In [ ]:
for i in range(len(data['capital-gain'])):
  if data['capital-gain'][i]!=0 and data['capital-loss'][i]!=0:
    print(i)
    break
print('No such entry')

In [ ]:
plt.hist(data['capital-gain'])
plt.show()

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(data[sel_cols],data['salaryT'],test_size=0.33,random_state=42)

In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred,target_names=['<=50K','>50K']))

In [ ]:
sns.distplot(data['age'])

In [ ]:
sns.histplot(data['age'],bins=10)

In [ ]:
data['age_slab'] = data['age']
for i in range(len(data['age_slab'])):
    a = data['age_slab'][i]
    if a<31 :
        data['age_slab'][i] = 1
    elif a>31 and a<41:
        data['age_slab'][i] = 2
    elif a>41 and a<51:
        data['age_slab'][i] = 3
    elif a>51 and a<71:
        data['age_slab'][i]=4
    elif a>=71:
        data['age_slab'][i] = 5

In [ ]:
data['age_slab'] = data['age']
for i in range(len(data['age_slab'])):
  a=data['age_slab'][i]
  if a<31:
    data['age_slab'][i]=1
  elif a>=31 and a<41:
    data['age_slab'][i]=2
  elif a>=41 and a<51:
    data['age_slab'][i]=3
  elif a>=51 and a<71:
    data['age_slab'][i]=4
  elif a>=71:
    data['age_slab'][i]= 5

In [ ]:
plt.hist(data[data['salary']=='<=50K']['age_slab'])
plt.show()
plt.hist(data[data['salary']=='>50K']['age_slab'])
plt.show()

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(data[sel_cols],data['salaryT'],test_size=0.33)
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred,target_names=['<=50K','>50K']))

In [ ]:
data[sel_cols].head()

In [ ]:
sns.distplot(data[data['salary']=='<=50K']['hours-per-week'])
plt.title('<=50K')
plt.show()
sns.distplot(data[data['salary']=='<=50K']['hours-per-week'])
plt.title('>50K')
plt.show()

In [ ]:
sns.distplot(data[data['salary']=='<=50K']['education-num'])
plt.title('<=50K')
plt.show()
sns.distplot(data[data['salary']=='<=50K']['education-num'])
plt.title('>50K')
plt.show()

In [ ]:
print(data.groupby(by=['salary'])['hours-per-week'].mean())
data.groupby(by=['salary'])['hours-per-week'].median()

In [ ]:
data.groupby(by=['salary'])['racet'].mean()

In [ ]:
data[sel_cols].head()

In [ ]:
df=pd.DataFrame(data.groupby(['race'])['salary'].value_counts())
df

In [ ]:
data.groupby(['sex'])['salary'].value_counts()

In [ ]:
X_train.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(classification_report(y_pred,y_test))

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
model = SGDClassifier()
scaler = StandardScaler()
v = scaler.fit_transform(X_train)
model.fit(v,y_train)
v2 = scaler.fit_transform(X_test)
y_pred = model.predict(v2)
print(classification_report(y_pred,y_test))




In [ ]:

from sklearn.linear_model import LogisticRegressionCV
model=LogisticRegressionCV(cv=5)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(classification_report(y_pred,y_test))


In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
model=KNeighborsClassifier()
model.fit(scaler.fit_transform(X_train),y_train)
y_pred=model.predict(scaler.fit_transform(X_test))
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.svm import SVC
model = SVC()
scaler  = StandardScaler()
model.fit(scaler.fit_transform(X_train),y_train)
y_pred = model.predict(scaler.fit_transform(X_test))
print(classification_report(y_test,y_pred))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.ensemble   import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))
print('Confusion matrix\n',confusion_matrix(y_test,y_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model = QuadraticDiscriminantAnalysis()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(classification_report(y_test,y_pred))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimator = [int(i) for i in range(80,100)]
learning_rate = [int(i) for i in range(-5,5)]
alg=['SAMME', 'SAMME.R']
random_grid={'n_estimators':n_estimator,
             'learning_rate':learning_rate,
             'algorithm':alg
             }
print(random_grid)